In [ ]:
# Definitions and imports
import time

In [ ]:
# Read input from the benchmark file
def read_input(file_path):
    with open(file_path, 'r') as file:
        num_objects = int(file.readline().strip())
        bin_capacity = int(file.readline().strip())
        weights = [int(line.strip()) for line in file]

    return num_objects, bin_capacity, weights

# Example usage:
#file_path = 'benchmark/BPP_100_100_0.1_0.7_0.txt'
file_path = 'benchmark/unordered_benchmark50.txt'
num_objects, bin_capacity, weights = read_input(file_path)

**First attempt:** Basic Best fit algorithm

In [ ]:
def best_fit(weights, bin_capacity):
    bins = [bin_capacity]  # Initialize the first bin with full capacity
    num_bins = 1

    for weight in weights:
        best_bin_index = -1
        min_space_left = float('inf')

        # Find the bin with the best fit for the current item
        for i, bin_space_left in enumerate(bins):
            if weight <= bin_space_left < min_space_left:
                best_bin_index = i
                min_space_left = bin_space_left

        if best_bin_index == -1:
            # No bin found, create a new bin
            bins.append(bin_capacity - weight)
            num_bins += 1
        else:
            # Assign the item to the best-fit bin
            bins[best_bin_index] -= weight

    return num_bins


In [ ]:
# Measure execution time
start_time = time.time()

# Run Best Fit algorithm
optimal_bins = best_fit(weights, bin_capacity)

# Calculate execution time
execution_time = time.time() - start_time

print(f"Optimal number of bins: {optimal_bins}")
print(f"Execution time: {execution_time} seconds")

Optimal number of bins: 24
Execution time: 0.0004107952117919922 seconds


**Second attempt:** Combine the Best fit to a re-organisation algorithm to minimise the lost-space per bins for smaller items and potentially minimise the number of bins.

Details --> It involves an additional step after the initial placement of items using the Best Fit algorithm. When the sum of the wasted space in all bins surpasses a certain threshold (in this case, the minimum weight of the remaining objects), the reorganization is triggered. During reorganization, the algorithm examines each bin, and if the last item in a bin has lost space (remaining space below the minimum weight) that is considered non-optimal, the algorithm removes this item. Subsequently, it identifies an item from the remaining pool of objects that maximizes the lost space and places it into the bin. This iterative process is designed to improve the space utilization of the bins and, potentially, reduce the overall number of bins used, as it alternates between the Best Fit algorithm and a worst-case placement strategy for lost space maximization.

In [ ]:
def reorganize_bins(bins, weights):
    # Calculate the minimum weight of the remaining objects
    min_remaining_weight = min(weights)

    # Calculate the sum of wasted space in all bins
    wasted_space_sum = sum([bin_space_left for bin_space_left in bins if bin_space_left < min_remaining_weight])
    #print(f" min= {min_remaining_weight} | waste= , {wasted_space_sum}")

    # Check if reorganization is needed
    if wasted_space_sum >= min_remaining_weight:
        removed_items = []  # To store the removed items

        # Sort bins by remaining space in descending order
        sorted_bins = sorted(enumerate(bins), key=lambda x: x[1], reverse=True)

        # Iterate through the bins and try to reorganize
        for i, bin_space_left in sorted_bins:
            if bin_space_left >= min_remaining_weight:
                # Check if the last item in the bin has lost space less than min weight
                last_item_lost_space = bin_space_left - min_remaining_weight
                if last_item_lost_space < min_remaining_weight:
                    # Remove the last item from the bin and store it
                    removed_items.append((i, last_item_lost_space))

        # Place removed items using Best Fit
        for i, lost_space in removed_items:
            # Find the item with the maximum weight to maximize lost space
            max_weight_item_index = max(range(len(weights)), key=lambda x: weights[x])

            # Place the item in the current bin
            bins[i] -= weights[max_weight_item_index]
            weights[max_weight_item_index] = 0  # Mark the item as placed

        return True  # Reorganization successful

    return False  # No reorganization performed

In [ ]:
def best_fit_org(weights, bin_capacity):
    bins = [bin_capacity]  # Initialize the first bin with full capacity
    num_bins = 1

    for weight in weights:
        best_bin_index = -1
        min_space_left = float('inf')

        # Find the bin with the best fit for the current item
        for i, bin_space_left in enumerate(bins):
            if weight <= bin_space_left < min_space_left:
                best_bin_index = i
                min_space_left = bin_space_left

        if best_bin_index == -1:
            # No bin found, create a new bin
            bins.append(bin_capacity - weight)
            num_bins += 1
        else:
            # Assign the item to the best-fit bin
            bins[best_bin_index] -= weight

        # Perform reorganization if needed
        check =  reorganize_bins(bins, weights)
        #print(check)
        if check:
            # Update the number of bins after reorganization
            num_bins = len([bin_space_left for bin_space_left in bins if bin_space_left < bin_capacity])

    return num_bins

In [ ]:
# Measure execution time
start_time = time.time()

# Run Best Fit algorithm
optimal_bins = best_fit_org(weights, bin_capacity)

# Calculate execution time
execution_time = time.time() - start_time

print(f"Optimal number of bins: {optimal_bins}")
print(f"Execution time: {execution_time} seconds")

Optimal number of bins: 24
Execution time: 0.001065969467163086 seconds


**Third attempts:** Annealing simulation to refine Best Fit solutions

In [ ]:
import random
import math
import time

def best_fit(weights, bin_capacity):
    bins = [bin_capacity]  # Initialize the first bin with full capacity
    num_bins = 1

    for weight in weights:
        best_bin_index = -1
        min_space_left = float('inf')

        # Find the bin with the best fit for the current item
        for i, bin_space_left in enumerate(bins):
            if weight <= bin_space_left < min_space_left:
                best_bin_index = i
                min_space_left = bin_space_left

        if best_bin_index == -1:
            # No bin found, create a new bin
            bins.append(bin_capacity - weight)
            num_bins += 1
        else:
            # Assign the item to the best-fit bin
            bins[best_bin_index] -= weight

    return num_bins

def reorganize_bins_simulated_annealing(bins, weights):
    # Perform a simple swap between two random bins
    bin_indices = list(range(len(bins)))

    if len(bin_indices) >= 2:
        # Choose two random bins
        i, j = random.sample(bin_indices, 2)

        # Swap their contents
        bins[i], bins[j] = bins[j], bins[i]

        return True  # Reorganization successful

    return False  # No reorganization performed

def simulated_annealing_best_fit(weights, bin_capacity, initial_temperature=1000, cooling_rate=0.95, num_iterations=100):
    current_solution = best_fit(weights, bin_capacity)
    current_cost = current_solution  # Assuming the number of bins is the cost in this case

    best_solution = current_solution
    best_cost = current_cost

    temperature = initial_temperature

    for _ in range(num_iterations):
        # Generate a neighbor solution by applying reorganization
        neighbor_solution = best_fit(weights, bin_capacity)
        neighbor_cost = neighbor_solution

        # Decide whether to accept the neighbor solution
        if neighbor_cost < current_cost or random.uniform(0, 1) < math.exp((current_cost - neighbor_cost) / temperature):
            current_solution = neighbor_solution
            current_cost = neighbor_cost

        # Update the best solution if needed
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Cool down the temperature
        temperature *= cooling_rate

    return best_solution



start_time = time.time()

optimal_bins_simulated_annealing = simulated_annealing_best_fit(weights, bin_capacity)

execution_time_simulated_annealing = time.time() - start_time

start_time = time.time()

optimal_bins_best_fit = best_fit(weights, bin_capacity)

execution_time_best_fit = time.time() - start_time

print(f"Optimal number of bins (Simulated Annealing): {optimal_bins_simulated_annealing}")
print(f"Execution time (Simulated Annealing): {execution_time_simulated_annealing} seconds")

print(f"\nOptimal number of bins (Best Fit): {optimal_bins_best_fit}")
print(f"Execution time (Best Fit): {execution_time_best_fit} seconds")


Optimal number of bins (Simulated Annealing): 24
Execution time (Simulated Annealing): 0.021287202835083008 seconds

Optimal number of bins (Best Fit): 24
Execution time (Best Fit): 0.00027489662170410156 seconds
